## 데이터 불러오기

In [20]:
import numpy as np
import pandas as pd

In [21]:
classlist = ['isc_2_1','isc_3_1','isc_3_3', # 0, 1, 2
             'nsc_1_1','nsc_1_2','nsc_2_5', # 3, 4, 5 
             'sdc_1_1','sdc_2_1',           # 6, 7
             'sic_2_6','sic_4_2']           # 8, 9
# ----------------------- 데이터 선택------------------------------
NUMBER = 2;
# ----------------------- 데이터 선택------------------------------

choosefile = classlist[NUMBER]
data_address = 'https://raw.githubusercontent.com/hanyeob/fine_dust/master/csv%20files/' + choosefile +".csv"

In [22]:
dataset = pd.read_csv(data_address)
dataset[0:5]

,tt,0.3_outdoor,0.5_outdoor,1.0_outdoor,3.0_outdoor,5.0_outdoor,10.0_outdoor,0.3_classroom,0.5_classroom,1.0_classroom,3.0_classroom,5.0_classroom,10.0_classroom,temp_outdoor,RH_outdoor,AH_outside,CO2_outdoor,temp_classroom,RH_classroom,AH_classroom,CO2_classroom,temp_meteo_425,RH_meteo_425,WD_meteo_425,WS_meteo_425,temp_meteo_889,RH_meteo_889,WD_meteo_889,WS_meteo_889,PM10_air_outdoor,PM2.5_air_outdoor,temp_air_outdoor,RH_air_outdoor,AH_air_outdoor,CO2_air_outdoor,VOCS_air_outdoor,Noise_air_outdoor,PM10_air_classroom,PM2.5_air_classroom,temp_air_classroom,RH_air_classroom,AH_air_classroom,CO2_air_classroom,VOCS_air_classroom,Noise_air_classroom,ΔT
0,11-11-2019 16:57:27,169956900,18550180,1706360,132509,41696,3887,104509200,11037100,1027208,95053,39929,11661,14.9,61.0,NaN,491.0,21.6,43,NaN,761.0,13.1,76.9,84.3,0.0,14.4,64.9,227.4,2.6,34.0,17.0,16.7,53.0,NaN,660.0,189.0,53,10.0,6.0,25.0,35.0,NaN,640.0,36.0,52.0,8.6
1,11-11-2019 16:58:27,167429700,18210950,1639576,110954,34629,1767,102582700,10870320,1073145,98587,29682,4594,14.8,61.0,NaN,471.0,21.6,43,NaN,762.0,13.2,76.7,192.1,1.5,14.4,65.2,249.8,2.2,40.0,18.0,16.6,54.0,NaN,637.0,185.0,53,9.0,5.0,24.9,35.0,NaN,597.0,23.0,52.0,8.6
2,11-11-2019 16:59:27,168108800,18259720,1732509,109894,41343,2473,99188700,10577380,1013428,93286,34629,9541,14.9,61.0,NaN,488.0,21.6,43,NaN,762.0,13.1,76.4,271.1,0.3,14.4,65.3,242.3,2.1,32.0,15.0,16.8,53.0,NaN,661.0,257.0,53,5.0,3.0,24.9,35.0,NaN,597.0,27.0,52.0,8.3
3,11-11-2019 17:00:27,169491500,18756540,1728269,106360,36396,707,100808800,10813430,1051237,87279,30035,6360,14.9,61.0,NaN,502.0,21.6,43,NaN,763.0,13.1,76.6,220.1,0.7,14.4,64.9,256.2,2.1,22.0,12.0,17.0,53.0,NaN,664.0,250.0,53,5.0,3.0,24.8,35.0,NaN,632.0,21.0,52.0,8.1
4,11-11-2019 17:01:27,168011700,19518380,1830035,115194,36042,2120,99027210,10662540,1013781,73852,26148,4594,15.0,61.0,NaN,487.0,21.6,43,NaN,762.0,13.1,76.7,238.1,0.2,14.4,64.5,246.8,2.1,32.0,19.0,17.0,53.0,NaN,656.0,195.0,53,6.0,4.0,24.7,35.0,NaN,614.0,22.0,53.0,8.0


In [23]:
dataset.columns

Index(['tt', '0.3_outdoor', '0.5_outdoor', '1.0_outdoor', '3.0_outdoor',
       '5.0_outdoor', '10.0_outdoor', '0.3_classroom', '0.5_classroom',
       '1.0_classroom', '3.0_classroom', '5.0_classroom', '10.0_classroom',
       'temp_outdoor', 'RH_outdoor', 'AH_outside', 'CO2_outdoor',
       'temp_classroom', 'RH_classroom', 'AH_classroom', 'CO2_classroom',
       'temp_meteo_425', 'RH_meteo_425', 'WD_meteo_425', 'WS_meteo_425',
       'temp_meteo_889', 'RH_meteo_889', 'WD_meteo_889', 'WS_meteo_889',
       'PM10_air_outdoor', 'PM2.5_air_outdoor', 'temp_air_outdoor',
       'RH_air_outdoor', 'AH_air_outdoor', 'CO2_air_outdoor',
       'VOCS_air_outdoor', 'Noise_air_outdoor', 'PM10_air_classroom',
       'PM2.5_air_classroom', 'temp_air_classroom', 'RH_air_classroom',
       'AH_air_classroom', 'CO2_air_classroom', 'VOCS_air_classroom',
       'Noise_air_classroom', 'ΔT'],
      dtype='object')

In [24]:
dataset = dataset[['tt', '0.3_outdoor', '0.5_outdoor', '1.0_outdoor', '3.0_outdoor',
       '5.0_outdoor', '10.0_outdoor', '0.3_classroom', '0.5_classroom',
       '1.0_classroom', '3.0_classroom', '5.0_classroom', '10.0_classroom',
       'CO2_outdoor','CO2_classroom']]

In [25]:
dataset['tt'] = pd.to_datetime(dataset['tt'], format="%d-%m-%Y %H:%M:%S")

## NuN값 처리

In [26]:
dataset.isnull().sum()

tt                  0
0.3_outdoor         0
0.5_outdoor         0
1.0_outdoor         0
3.0_outdoor         0
5.0_outdoor         0
10.0_outdoor        0
0.3_classroom       0
0.5_classroom       0
1.0_classroom       0
3.0_classroom       0
5.0_classroom       0
10.0_classroom      0
CO2_outdoor         0
CO2_classroom     130
dtype: int64

In [27]:
for i in reversed(range(0,len(dataset))): 
    if(np.isnan(dataset.loc[i,'CO2_classroom'])):
      if(i+1 < len(dataset)):
        dataset.loc[i,'CO2_classroom'] = dataset.loc[i+1,'CO2_classroom']

for i in range(0,len(dataset)): 
    if(np.isnan(dataset.loc[i,'CO2_classroom'])):
      if(i > 0):
        dataset.loc[i,'CO2_classroom'] = dataset.loc[i-1,'CO2_classroom']

## 미세먼지, 초미세먼지 단위 맞춤 (Scaling)

In [28]:
dataset['0.3_outdoor_scaled'] = dataset['0.3_outdoor'] * 0.3 ** 3
dataset['0.5_outdoor_scaled'] = dataset['0.5_outdoor'] * 0.5 ** 3
dataset['1.0_outdoor_scaled'] = dataset['1.0_outdoor'] * 1.0 ** 3
dataset['3.0_outdoor_scaled'] = dataset['3.0_outdoor'] * 3.0 ** 3
dataset['5.0_outdoor_scaled'] = dataset['5.0_outdoor'] * 5.0 ** 3
dataset['10.0_outdoor_scaled'] = dataset['10.0_outdoor'] * 10.0 ** 3

dataset['0.3_classroom_scaled'] = dataset['0.3_classroom'] * 0.3 ** 3
dataset['0.5_classroom_scaled'] = dataset['0.5_classroom'] * 0.5 ** 3
dataset['1.0_classroom_scaled'] = dataset['1.0_classroom'] * 1.0 ** 3
dataset['3.0_classroom_scaled'] = dataset['3.0_classroom'] * 3.0 ** 3
dataset['5.0_classroom_scaled'] = dataset['5.0_classroom'] * 5.0 ** 3
dataset['10.0_classroom_scaled'] = dataset['10.0_classroom'] * 10.0 ** 3

## 미세먼지/초미세먼지(내부,외부) 데이터 통합

In [29]:
dataset['PM2.5_outdoor'] = np.sum(dataset.loc[:,'0.3_outdoor_scaled':'1.0_outdoor_scaled'], axis=1)
dataset['PM10_outdoor'] =  np.sum(dataset.loc[:,'3.0_outdoor_scaled':'10.0_outdoor_scaled'], axis=1)

dataset['PM2.5_classroom'] = np.sum( dataset.loc[:,'0.3_classroom_scaled':'1.0_classroom_scaled'], axis=1)
dataset['PM10_classroom'] = np.sum( dataset.loc[:,'3.0_classroom_scaled':'10.0_classroom_scaled'], axis=1)

In [30]:
# CO2_fig = go.Figure()
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_outdoor'],mode='lines',name='CO2_outdoor'))
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_classroom'],mode='lines',name='CO2_classroom'))

In [31]:
# PM_fig = go.Figure()
# from plotly.subplots import make_subplots
# PM_fig = make_subplots(specs=[[{"secondary_y": True}]])

# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM2.5_outdoor'],line=dict(color='red'), mode='lines',name='PM2.5_outdoors'))
# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM10_outdoor'],line=dict(color='blue'),mode='lines',name='PM10_outdoors'),secondary_y=True,)
# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM2.5_classroom'],line=dict(color='orange'),mode='lines',name='PM2.5_classroom'))
# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM10_classroom'],line=dict(color='skyblue'),mode='lines',name='PM10_classroom'),secondary_y=True,)

# PM_fig.update_yaxes(title_text="PM 2.5", secondary_y=False)
# PM_fig.update_yaxes(title_text="PM10", secondary_y=True)

## 교실 내 변화량 column생성

In [32]:
for i in range(1,len(dataset)) :
    dataset.loc[i,'PM2.5_classroom_change'] = dataset.loc[i,'PM2.5_classroom'] - dataset.loc[i-1,'PM2.5_classroom']
    dataset.loc[i,'PM10_classroom_change'] = dataset.loc[i,'PM10_classroom'] - dataset.loc[i-1,'PM10_classroom']
    dataset.loc[i,'CO2_classroom_change'] = dataset.loc[i,'CO2_classroom'] - dataset.loc[i-1,'CO2_classroom']

dataset.loc[0,'PM2.5_classroom_change'] =dataset.loc[1,'PM2.5_classroom_change']
dataset.loc[0,'PM10_classroom_change']  =dataset.loc[1,'PM10_classroom_change'] 
dataset.loc[0,'CO2_classroom_change']   =dataset.loc[1,'CO2_classroom_change']  

In [33]:
# CO2_fig = go.Figure()
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_outdoor'],mode='lines',name='CO2_outdoor'))
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_classroom'],mode='lines',name='CO2_classroom'))

# 스케일링 (일반화)


In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaled_data = scaler.fit_transform(dataset[
                ['CO2_outdoor','PM2.5_outdoor','PM10_outdoor',
               'CO2_classroom','PM2.5_classroom','PM10_classroom',
               'CO2_classroom_change','PM2.5_classroom_change','PM10_classroom_change']
              ])
scaled_data = pd.DataFrame(data=scaled_data,columns=['CO2_outdoor','PM2.5_outdoor','PM10_outdoor',
               'CO2_classroom','PM2.5_classroom','PM10_classroom',
               'CO2_classroom_change','PM2.5_classroom_change','PM10_classroom_change'])
scaled_data.insert(loc=0,column='tt',value=dataset['tt'])
scaled_data

,tt,CO2_outdoor,PM2.5_outdoor,PM10_outdoor,CO2_classroom,PM2.5_classroom,PM10_classroom,CO2_classroom_change,PM2.5_classroom_change,PM10_classroom_change
0,2019-11-11 16:57:27,0.676260,0.007972,-0.270523,0.330962,1.336983,-0.030959,0.039168,-0.126862,-0.544022
1,2019-11-11 16:58:27,0.194994,-0.030506,-0.405798,0.333822,1.322819,-0.217368,0.039168,-0.126862,-0.544022
2,2019-11-11 16:59:27,0.604070,-0.005053,-0.348576,0.333822,1.223942,-0.094888,-0.005674,-0.896913,0.356791
3,2019-11-11 17:00:27,0.940956,0.015593,-0.442138,0.336681,1.282360,-0.183377,0.039168,0.532914,-0.258455
4,2019-11-11 17:01:27,0.580007,0.049652,-0.381496,0.333822,1.227433,-0.242432,-0.050517,-0.497404,-0.172618
...,...,...,...,...,...,...,...,...,...,...
8348,2019-11-20 12:31:13,-0.791601,-0.839949,-0.380980,3.488260,-0.578397,0.360849,-1.395790,0.270054,0.514751
8349,2019-11-20 12:32:13,-0.767538,-1.015653,-0.386074,3.462521,-0.540244,0.396435,-0.409256,0.348692,0.103385
8350,2019-11-20 12:33:13,-0.719411,-1.023306,-0.386282,3.433922,-0.473433,0.967689,-0.454099,0.609205,1.665556
8351,2019-11-20 12:34:13,-0.647221,-0.845807,-0.424835,3.391024,-0.447163,1.211711,-0.678311,0.240674,0.711246


In [35]:
scaled_data

,tt,CO2_outdoor,PM2.5_outdoor,PM10_outdoor,CO2_classroom,PM2.5_classroom,PM10_classroom,CO2_classroom_change,PM2.5_classroom_change,PM10_classroom_change
0,2019-11-11 16:57:27,0.676260,0.007972,-0.270523,0.330962,1.336983,-0.030959,0.039168,-0.126862,-0.544022
1,2019-11-11 16:58:27,0.194994,-0.030506,-0.405798,0.333822,1.322819,-0.217368,0.039168,-0.126862,-0.544022
2,2019-11-11 16:59:27,0.604070,-0.005053,-0.348576,0.333822,1.223942,-0.094888,-0.005674,-0.896913,0.356791
3,2019-11-11 17:00:27,0.940956,0.015593,-0.442138,0.336681,1.282360,-0.183377,0.039168,0.532914,-0.258455
4,2019-11-11 17:01:27,0.580007,0.049652,-0.381496,0.333822,1.227433,-0.242432,-0.050517,-0.497404,-0.172618
...,...,...,...,...,...,...,...,...,...,...
8348,2019-11-20 12:31:13,-0.791601,-0.839949,-0.380980,3.488260,-0.578397,0.360849,-1.395790,0.270054,0.514751
8349,2019-11-20 12:32:13,-0.767538,-1.015653,-0.386074,3.462521,-0.540244,0.396435,-0.409256,0.348692,0.103385
8350,2019-11-20 12:33:13,-0.719411,-1.023306,-0.386282,3.433922,-0.473433,0.967689,-0.454099,0.609205,1.665556
8351,2019-11-20 12:34:13,-0.647221,-0.845807,-0.424835,3.391024,-0.447163,1.211711,-0.678311,0.240674,0.711246


In [36]:
scaled_data.isnull().sum()

tt                        0
CO2_outdoor               0
PM2.5_outdoor             0
PM10_outdoor              0
CO2_classroom             0
PM2.5_classroom           0
PM10_classroom            0
CO2_classroom_change      0
PM2.5_classroom_change    0
PM10_classroom_change     0
dtype: int64

In [37]:
scaled_data.to_csv(choosefile+'_for_predict.csv',index=False)